$\textbf{Контекст:}$ сейчас март 2018. Команда Olist разработала новый улучшенный алгоритм рекомендации товаров.

$\textbf{Задача:}$ провести А/Б тестирование, чтобы выяснить, стоит ли запускать новый алгоритм. В качестве целевой метрики выберем ARPU - average revenue per user.

Импортируем необходимые библиотеки: 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans
from sqlalchemy import create_engine

Подключимся к нашей базе данных, и получим датафрейм с номером покупателя и суммой заказа. Заказы рассматриваем за годовой период начиная с февраля 2017 года. Эти данные нам нужны чтобы оценить протестировать наш алгоритм сплита пользователей, и оценить MDE и для АА тестирования.

In [ ]:
username = "postgres"
host = "localhost"
port = "5432"
database = "olist_db"
password = "my_password"

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

query = """SELECT price as revenue, customer_id
FROM order_items oi LEFT JOIN orders o USING (order_id)
WHERE o.order_status = 'delivered' AND DATE_TRUNC('month', o.order_purchase_timestamp)::DATE = '2018-02-01';
"""

df = pd.read_sql_query(query, engine)
df.head()


,revenue,customer_id
0,19.9,8ab97904e6daea8866dbdbc4fb7aad2c
1,9.5,72ae281627a6102d9b3718528b420f8a
2,109.9,3a874b4d4c4b6543206ff5d89287f0c3
3,30.1,761df82feda9778854c6dafdaeb567e4
4,84.9,d9ef95f98d8da3b492bb8c0447910498


Для удобства создадим новый вспомогательный датасет.

In [21]:
user_rev = df.groupby('customer_id')['revenue'].sum().reset_index()
user_rev.head()

,customer_id,revenue
0,00066ccbe787a588c52bd5ff404590e3,199.60
1,001450ebb4a77efb3d68be5f7887cb1e,37.64
2,0029cdf064769cabdf3186b54d068c99,459.99
3,00380c010de38d578d02117f6d0b88e7,49.00
4,003cbe6a43560a8b6fd2c07531257c2d,13.65


Разделим пользователей на 2 равные группы случайно. 

In [22]:
np.random.seed(42)  

user_rev['group'] = np.where(np.random.rand(len(user_rev)) > 0.5, 'A', 'B')

user_rev.groupby('group').agg({'customer_id':'count', 'revenue':'mean'})


,customer_id,revenue
group,,
A,3270,124.991673
B,3285,127.158100


Как и ожидалось, SRM не обнаружено (используем критерий хи-квадрат).

In [23]:
obs = user_rev.group.value_counts().values
stats.chisquare(obs)


Power_divergenceResult(statistic=np.float64(0.034324942791762014), pvalue=np.float64(0.8530173309245255))

Выборки большие, так что нормальность данных не играет роли при использовании t-test. На всякий случай используем Манна-Уитни. Как и ожидалось, различия в ARPU между группами не найдено.

In [24]:
A = user_rev.loc[user_rev['group']=='A', 'revenue']
B = user_rev.loc[user_rev['group']=='B', 'revenue']

print(stats.ttest_ind(A, B, equal_var=False))
print(stats.mannwhitneyu(A, B, alternative='two-sided'))



TtestResult(statistic=np.float64(-0.5357771433998493), pvalue=np.float64(0.5921310984696642), df=np.float64(6434.701963593819))
MannwhitneyuResult(statistic=np.float64(5339523.5), pvalue=np.float64(0.681397986237573))


Возьмем уровень значимости 0.05, мощность 0.8, а размер выборки, дисперсию и среднее оценим по уже имеющимся данным. Сосчитаем effect size и MDE. Можно запускать тестирование и без этого, но проделаем на всякий случай.

In [42]:
arpu = user_rev['revenue'].mean()  

sample_size = np.floor(len(user_rev.customer_id)/(12*2))
mu = user_rev['revenue'].mean()
sigma = user_rev['revenue'].std()
power = 0.8     
alpha=0.05

effect_size = tt_ind_solve_power(nobs1=sample_size, alpha=alpha, power=power, alternative='two-sided')
mde = effect_size * sigma / mu
print(f"MDE = {mde:.3f}")
print(f"Это значит, что чтобы t-test замечал разницу в ARPU с данной мощностью, уровнем значимости, дисперсией, средним и размером выборки, нужно, чтобы она изменилась хотя бы на {mde*arpu:.3f}")



MDE = 0.312
Это значит, что чтобы t-test замечал разницу в ARPU с данной мощностью, уровнем значимости, дисперсией, средним и размером выборки, нужно, чтобы она изменилась хотя бы на 39.331


Предположим, мы запустили процесс АБ тестирования на месяц. Для этого искусственно увеличим выручку в группе B на 10%.

In [26]:
query = """SELECT price as revenue, customer_id
FROM order_items oi LEFT JOIN orders o USING (order_id)
WHERE o.order_status = 'delivered' AND DATE_TRUNC('month', o.order_purchase_timestamp)::DATE = '2018-03-01';
"""

df_after = pd.read_sql_query(query, engine)
user_rev_after = df_after.groupby('customer_id')['revenue'].sum().reset_index()

np.random.seed(42)  

user_rev_after['group'] = np.where(np.random.rand(len(user_rev_after)) > 0.5, 'A', 'B')

user_rev_after.groupby('group').agg({'customer_id':'count', 'revenue':'mean'})

user_rev_after.loc[user_rev_after['group']=='B', 'revenue'] *= 1.1



Аналогично, SRM не обнаружено.

In [27]:
obs = user_rev_after.group.value_counts().values
stats.chisquare(obs)


Power_divergenceResult(statistic=np.float64(0.31543624161073824), pvalue=np.float64(0.5743632920067822))

Как мы можем заметить, изменения в ARPU статистически значимы.

In [28]:
A = user_rev_after.loc[user_rev_after['group']=='A', 'revenue']
B = user_rev_after.loc[user_rev_after['group']=='B', 'revenue']

print(stats.ttest_ind(A, B, equal_var=False))
print(stats.mannwhitneyu(A, B, alternative='two-sided'))

TtestResult(statistic=np.float64(-2.9055485113117876), pvalue=np.float64(0.003677755425943748), df=np.float64(6899.3204233983015))
MannwhitneyuResult(statistic=np.float64(5688216.5), pvalue=np.float64(1.767455260633805e-07))


Относительный прирост:

In [35]:
(B.mean() - A.mean())/A.mean()

np.float64(0.10067809975351978)

Примерно 10%.

Построим доверительный интервал для разности средних:

In [ ]:
cm = CompareMeans(DescrStatsW(B), DescrStatsW(A))
cm.tconfint_diff(usevar='unequal', alpha=0.05)  

(np.float64(4.457441078559084), np.float64(22.945763843809758))

Вердикт: раскатываем фичу.